# Easy maps with folium#

Folium builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the Leaflet.js library. It's essentially a Python wrapper around the excellent Leaflet.js mapping library. Manipulate your data in Python, then visualize it in on a Leaflet map via Folium.

The library has a number of built-in tilesets from OpenStreetMap, MapQuest Open, MapQuest Open Aerial, Mapbox, and Stamen, and supports custom tilesets with Mapbox or Cloudmade API keys. Folium supports both GeoJSON and TopoJSON overlays, as well as the binding of data to those overlays to create choropleth maps with color-brewer color schemes.

In [ ]:
import folium
from IPython.display import HTML
import json
import pandas as pd
from shapely.geometry import Polygon
from math import cos, pi

In [ ]:
# Utility function to embed maps directly in the notebook
def inline_map(m, width=1200, height=500, input_html=False):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    if not input_html:
        m._build_map()
        srcdoc = m.HTML.replace('"', '&quot;')
    else:
        srcdoc = m.replace('"', '&quot;')
    return HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))

### Displaying a map###

In [ ]:
# Initialize a folium map simply by specifying the lat/lon to be centered at
berlin_loc = [52.51, 13.42]
map_osm = folium.Map(location=berlin_loc)
inline_map(map_osm)

Available tiles for map:
- "OpenStreetMap"
- "MapQuest Open"
- "MapQuest Open Aerial"
- "Mapbox Bright" (Limited levels of zoom for free tiles)
- "Mapbox Control Room" (Limited levels of zoom for free tiles)
- "Stamen Terrain"
- "Stamen Toner"
- "Cloudmade" (Must pass API key)
- "Mapbox" (Must pass API key)

In [ ]:
# Some additional init parameters are the zoom level (higher --> more zoom) and tiles
map_mqo = folium.Map(location=berlin_loc, zoom_start=13, tiles='MapQuest Open')
inline_map(map_mqo)

In [ ]:
map_mbb = folium.Map(location=berlin_loc, tiles='Mapbox Bright')
inline_map(map_mbb)

### Markers###

Folium allows to place markers on the map easily. To find out the lat/lon of a given location, a (forward) geocoder is needed. One can use Google Maps, or the following utility function to enable lat/lon popovers:

In [ ]:
map0 = folium.Map(location=[52.499, 13.335], zoom_start=16, tiles='MapQuest Open')
map0.lat_lng_popover()
inline_map(map0)

In [ ]:
map1 = folium.Map(location=[52.499, 13.335], zoom_start=16, tiles='MapQuest Open')

# Simple marker
map1.simple_marker([52.5003, 13.3366], popup='Augsburger Straße')

# Customization. Icon markers in http://getbootstrap.com/components/
map1.simple_marker([52.5007,13.3349], popup='Eislebenerstr. 4', marker_icon='asterisk', marker_color='red')

# Circle marker -- radius: circle radius, in pixels
map1.circle_marker([52.4973, 13.3297], popup='Gerhart-Hauptmann-Anlage', 
                   radius=100, line_color='orange', fill_color='green', fill_opacity=0.3)

# Polygon marker with N sides
map1.polygon_marker([52.5018, 13.3430], popup='Wittenbergplatz', num_sides=4, radius=30, rotation=45)
inline_map(map1)

### Exercises

1. Using the lat-lon popover above, find out the approximate lat/lon of Rio de Janeiro. Compare the coordinates with other Internet sources. 
2. Display a map centered on Rio de Janeiro and with a zoom level such that the full extension of Brazil is displayed.
3. Display a map centered on Rio de Janeiro at an appropriate zoom to resolve the city and place a marker on the Rodrigo de Freitas Lagoon.

### GeoJSON Overlays###

GeoJSON is an open standard format for encoding collections of simple geographical features along with their non-spatial attributes using JavaScript Object Notation. [Wikipedia GeoJSON](https://en.wikipedia.org/wiki/GeoJSON)

In [ ]:
# The geographical features are contained in a geojson file
# Source: https://github.com/m-hoerz/berlin-shapes
geo_path = r'berliner-bezirke.geojson'

map_osm.geo_json(reset=True,
                 geo_path=geo_path, 
                 fill_color='#ff8800', 
                 fill_opacity=0.4, 
                 line_color='#fff',
                 line_opacity=0.4,
                 line_weight=3)
inline_map(map_osm)

#### Binding data to GeoJSON Overlays

Visualizing geometries on a map is interesting, but even more so is to be able to show relations and magnitudes on top of it. Folium allows the binding of Pandas dataframes on GeoJSON geometries

In [ ]:
# GeoJSON data loads like any normal JSON data
geo_json = open(geo_path).read()
geo_data = json.loads(geo_json)

In [ ]:
geo_data.keys()

In [ ]:
geo_data['features'][0].keys()

In [ ]:
# The geometry is described by its type (Polygon) and coordinates
Polygon(geo_data['features'][0]['geometry']['coordinates'][0])

In [ ]:
# The boroughs' names are stored under properties/Name
geo_data['features'][0]['properties']['Name']

In [ ]:
# All boroughs' names
[gd['properties']['Name'] for gd in geo_data['features']]

In [ ]:
pd.DataFrame.from_csv?

In [ ]:
# Let's bind demographic data contained in a csv file. 
# The binding is made via the a common data column among the GeoJSON and the csv file

# Load data into dataframe, skip file header on initialization
data = pd.DataFrame.from_csv('berlin-data.csv', index_col=None, header=1)

# Let's create a new data column: population in thousands for simpler understanding and data viz
data['pop_thousands'] = data.apply(lambda row: round(row['population']/1000), axis=1)
data

In [ ]:
# We'll use the appropriate options of the geo_json function to produce the final map
map_osm.geo_json?

In [ ]:
# Display map binding geo_json and dataframe data -- using columns and key_on parameters
# This produces a choropleth map = Choro (Area/Region) + Pleth (quantity)
map_osm.geo_json(reset=True,
                 geo_path=geo_path,
                 data_out='data_rent.json',
                 data=data,
                 columns=['zone', 'avg_rent'],
                 key_on='feature.properties.Name',
                 fill_color='YlOrRd', 
                 fill_opacity=0.7, 
                 line_opacity=0.4,
                 line_color='white',
                 threshold_scale = [5,6,7,8,9,10],
                 legend_name="Berlin average rent per m2 (Euro)")
map_osm.create_map('berlin_rent.html')
inline_map(map_osm)

In [ ]:
# We can easily see which data is being bound
map_osm.json_data

### Exercises

1. Produce a similar map to the one above, but binding the percentage of foreigners per neighborhood in Berlin.
2. Produce a similar map to the one above, but binding the population per neighborhood in Berlin.
3. Produce a similar map to the one above, but binding the population density per neighborhood in Berlin. [Tip: use the geometry of the neighborhoods to extract ]